## Dataset en Voorverwerking

### Dataset 1 `(oecd_df)`
De titel van de dataset is *Green areas – Functioning Urban Areas (FUAs)* ([hier](https://sdmx.oecd.org/public/rest/data/OECD.CFE.EDS,DSD_FUA_ENV@DF_GREEN_AREA,1.0/.A..PT_LAR...?startPeriod=2017&dimensionAtObservation=AllDimensions&format=csvfilewithlabels) te downloaden). Deze dataset laat zien hoeveel groene gebieden er zijn in steden. Dit wordt gemeten met het aandeel van groen gebied ten opzichte van de totale landoppervlakte in procenten (`OBS_VALUE`). Voor de duidelijkheid is deze kolom hernoemd naar `Gemiddelde aandeel van groen in oppervlakte van stad`. De originele dataset bestond uit een waarde voor elke gemeten stad, wat resulteerde in 1129 waardes. Verder is deze dataset geaggregeerd tot een waarde per land, is de gemiddelde hoeveelheid groen per vierkante meter in alle landen 153.19, Daarnaast is de standaardafwijking is 64.58. En zijn de hoogste en laagste waarde respectievelijk 380.86 en 55.48.

### Dataset 2 `(stress_df)`
De titel van deze dataset is *What Is the World's Emotional Temperature? (2021-2022)* ([hier](https://news.gallup.com/interactives/248240/global-emotions.aspx) te downloaden). Deze dataset laat zien hoeveel procent van de mensen in een land de dag voordat het interview werd afgenomen stress heeft ondervonden. Dit bestaat uit een percentage van mensen die wel stress hebben ervaren (`YES`), een percentage van mensen die geen stress hebben ervaren (`NO`) en een percentage aan mensen die geen idee hebben of ze stress hebben ervaren of de vraag niet wilden beantwoorden (`DON'T KNOW/REFUSED`). Voor de duidelijkheid is de kolom `YES` hernoemd naar `Percentage mensen die Ja hebben geantwoord`, en is de kolom `NO` hernoemd naar `Percentage mensen die Nee hebben geantwoord`.
Verder is de gemiddelde waarde van `YES`: ~37.40% en de maximale waarde van “NO” is 89.00%. Voor “COUNTRY” is er geen statistiek mogelijk, naast dat er in 142 van de 196 landen in de wereld onderzoek is gedaan naar stress.

### Voorverwerking
`oecd_df` en `stress_df` bevatten geen gemeenschappelijke kolommen. Om de twee datasets toch samen te voegen moeten er wat aanpassingen aan beide datasets worden gedaan.

**`oecd_df`**: Voor deze dataset zijn er een aantal columns die niet gebruikt zullen worden verwijderd. Daarna is de tweeletterige landcode geëxtraheerd uit de `REF_AREA`-kolom. Hiermee kan deze dataset samengevoegd worden met een andere dataset, die landnamen en de bijbehorende twee- en drieletterige ISO-landcodes bevat. Deze is [hier](https://www.iban.com/country-codes) te downloaden. Nu bevat de dataset niet alleen het aandeel groen gebied ten opzichte van het totale landoppervlakte, maar ook de bijbehorende naam van het land waarin de gemeten stad zich bevindt. Nu kan de data geaggregeerd worden tot een waarde per land. Hiervoor is de `.groupby()`-functie gebruikt, net als de `.mean()`-methode om de gemiddelden voor alle steden in een land uit te rekenen. Deze dataset bevat ook per land de drieletterige ISO-code (kolom `3digit`), om de Plotly-functie `px.choropleth()` de juiste data op de juiste plekken op de wereldkaart te zetten.

**`stress_df`**: Voor deze dataset zijn alle waardes van een percentage (wat Python als tekst ziet) naar een getal omgezet. Dit stelt ons in staat om deze data te kunnen vergelijken met de data uit `oecd_df`. Verder is de kolom `DON'T KNOW/REFUSED` verwijderd, omdat we alleen geïnteresseerd waren in wie wel of geen stress hebben ondervonden. Als laatste is bij deze dataset ook de drieletterige ISO-code toegevoegd, zodat `px.choropleth()` gebruikt kan worden.

De twee datasets zijn uiteindelijk samengevoegd tot één dataset genaamd `merged_df`, op basis van de landsnaam.
genaamd `merged_df`, op basis van de landsnaam

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [2]:
oecd_df = pd.read_csv('original_oecd.csv')
#nutteloze kolommen verwijderen
kolommen_verwijderen = ['ACTION', 
                        'STRUCTURE', 
                        'STRUCTURE_ID', 
                        'STRUCTURE_NAME', 
                        'FREQ', 
                        'Frequency of observation', 
                        'MEASURE', 
                        'Measure', 
                        'UNIT_MEASURE', 
                        'Unit of measure', 
                        'PM_CONCENTRATION', 
                        'PM concentration level', 
                        'TIME_SEASON', 
                        'Time of the day and season', 
                        'TERRITORIAL_LEVEL', 
                        'Territorial level', 
                        'TIME_PERIOD', 
                        'Observation value', 
                        'OBS_STATUS', 
                        'UNIT_MULT', 
                        'Unit multiplier', 
                        'Time period', 
                        'Observation status', 
                        'DECIMALS', 
                        'Decimals']
oecd_df = oecd_df.drop(kolommen_verwijderen, axis=1)
# Edgecase: REF_AREA begint bij alle landen behalve Mexico met 2 letters
oecd_df['REF_AREA'] = oecd_df['REF_AREA'].str.replace('^MEX', 'MX', regex=True)
# sommige verkeerde landcodes worden hier vervangen door degene die correspondeert met iso_df
oecd_df['REF_AREA'] = oecd_df['REF_AREA'].str.replace('^UK', 'GB', regex=True)
oecd_df['REF_AREA'] = oecd_df['REF_AREA'].str.replace('^KOR', 'KR', regex=True)
oecd_df['REF_AREA'] = oecd_df['REF_AREA'].str.replace('^EL', 'GR', regex=True)
# kolom renamen voor duidelijkheid
oecd_df = oecd_df.rename(columns={'OBS_VALUE': 'Gemiddelde aandeel van groen in oppervlakte van stad'})

#Om de data per land te kunnen aggregeren, heb ik landsnamen als extra kolom Percentage mensen die Nee hebben geantwoorddig.
iso_df = pd.read_csv('23digit.csv')
iso_mapping = dict(zip(iso_df['2digit'], iso_df['Country']))

def get_country(geocode):
    country_code = ''.join(filter(str.isalpha, geocode))[:2]  # Verwijder alle niet-alfabetische tekens en neem de eerste twee letters
    return iso_mapping.get(country_code, 'Unknown')

oecd_df['Country'] = oecd_df['REF_AREA'].apply(get_country)
oecd_df = oecd_df.merge(iso_df, on='Country')
oecd_df = oecd_df.drop(['2digit', 'Numeric'], axis=1)


In [3]:
# nu gaan we de stress en iso df mergen met de country als key
stress_df = pd.read_csv('global-emotions-stress.csv')
def percentage_to_float(percentage):
    '''
    Zet een waarde met een % erachter om naar een float zodat het vergeleken kan worden
    '''
    return float(percentage.strip('%'))

stress_df['YES'] = stress_df['YES'].apply(percentage_to_float)
stress_df['NO'] = stress_df['NO'].apply(percentage_to_float)

stress_df = stress_df.drop(["DON'T KNOW/REFUSED"], axis=1)

stress_df = pd.merge(stress_df, iso_df, on='Country')
# stress df kolommen renamen
stress_df = stress_df.rename(columns={'YES': 'Percentage mensen die Ja hebben geantwoord'})
stress_df = stress_df.rename(columns={'NO': 'Percentage mensen die Nee hebben geantwoord'})

# nu gaan we de green df aggregeren

agg_df = oecd_df.groupby('Country')['Gemiddelde aandeel van groen in oppervlakte van stad'].mean().reset_index()
# tijd om de twee datasets te mergen
merged_df = pd.merge(agg_df, stress_df, on='Country')

# nutteloze kolommen weghalen
merged_df = merged_df.drop(['2digit', 'Numeric'], axis=1)
display(merged_df)

,Country,Gemiddelde aandeel van groen in oppervlakte van stad,Percentage mensen die Ja hebben geantwoord,Percentage mensen die Nee hebben geantwoord,3digit
0,Australia,38.431250,42.0,58.0,AUS
1,Austria,40.800000,32.0,68.0,AUT
2,Belgium,45.183333,39.0,60.0,BEL
3,Bulgaria,34.814286,23.0,74.0,BGR
4,Canada,46.457692,51.0,49.0,CAN
5,Chile,21.296154,37.0,62.0,CHL
6,Colombia,41.843396,47.0,53.0,COL
7,Croatia,43.633333,36.0,63.0,HRV
8,Cyprus,19.033333,55.0,45.0,CYP
9,Denmark,48.625000,25.0,75.0,DNK
